![](images/EscUpmPolit_p.gif "UPM")

# Course Notes for Learning Intelligent Systems

Department of Telematic Engineering Systems, Universidad Politécnica de Madrid, © Carlos A. Iglesias

# Exercises

# Table of Contents

* [Exercises](#Exercises)
	* [Exercise 1 - Sentiment Analysis on Movie Reviews](#Exercise-1---Sentiment-Analysis-on-Movie-Reviews)
	* [Exercise 2 - Spam classification](#Exercise-2---Spam-classification)
	* [Exercise 3 - Automatic essay classification](#Exercise-3---Automatic-essay-classification)

# Exercises

Here we propose several exercises, it is recommended to work only in one of them.

## Exercise 1 - Sentiment Analysis on Movie Reviews

You can try the exercise Exercise 2: Sentiment Analysis on movie reviews of Scikit-Learn https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html. 
Previously you should follow the installation instructions in the section Tutorial Setup.

## Exercise 2 - Spam classification

The classification of spam is a classical problem. [Here](http://zacstewart.com/2015/04/28/document-classification-with-scikit-learn.html) you can find a detailed example of how to do it using the datasets Enron-Spama and  SpamAssassin. You can try to test yourself the classification.

### Carga de datos

Primero cargamos los datos en un formato que pueda alimentar al algoritmo. Para un clasificador de SPAM sería útil tener una matriz bidimensional que contenga los cuerpos de los correos electrónicos en una columna y si es spam o no en otra.

In [1]:
import os
import numpy as np
import pandas as pd

NEWLINE = '\n'
SKIP_FILES = {'cmds'}


def read_files(path):
    for root, dir_names, file_names in os.walk(path):
        for path in dir_names:
            read_files(os.path.join(root, path))
        for file_name in file_names:
            if file_name not in SKIP_FILES:
                file_path = os.path.join(root, file_name)
                if os.path.isfile(file_path):
                    past_header, lines = False, []
                    f = open(file_path, encoding="latin-1")
                    for line in f:
                        if past_header:
                            lines.append(line)
                        elif line == NEWLINE:
                            past_header = True
                    f.close()
                    content = NEWLINE.join(lines)
                    yield file_path, content

Los correos electrónicos están separados por encabezado y cuerpo mediante una línea en blanco, por lo que ignoramos las líneas anteriores y cedemos el resto del correo electrónico.

Ahora con los datos de los cuerpos de los correos electrónicos obtenidos crearemos DataFrames y cuando necesitemos enviarlos al clasificador de SPAM lo convertimos en matriz Numpy.

In [2]:
def build_data_frame(path, classification):
    rows = []
    index = []
    for file_name, text in read_files(path):
        rows.append({'text': text, 'class': classification})
        index.append(file_name)

    data_frame = pd.DataFrame(rows, index=index)
    return data_frame

Esta función nos construirá un DataFrame a partir de todos los archivos en formato path. Incluirá el cuerpo del texto en una columna y la clase en otra. Cada fila será indexada por el nombre de archivo del correo electrónico correspondiente. Pandas le permite meter un DataFrame a otro DataFrame, es decir, construimos uno nuevo y agregamos al anterior repetidamente.

In [3]:
HAM = 'ham'
SPAM = 'spam'

SOURCES = [
    ('data/spam',        SPAM),
    ('data/easy_ham',    HAM),
    ('data/hard_ham',    HAM),
    ('data/beck-s',      HAM),
    ('data/farmer-d',    HAM),
    ('data/kaminski-v',  HAM),
    ('data/kitchen-l',   HAM),
    ('data/lokay-m',     HAM),
    ('data/williams-w3', HAM),
    ('data/BG',          SPAM),
    ('data/GP',          SPAM),
    ('data/SH',          SPAM)
]

data = pd.DataFrame({'text': [], 'class': []})
for path, classification in SOURCES:
    data = data.append(build_data_frame(path, classification))

data = data.reindex(np.random.permutation(data.index))

In [4]:
data

,text,class


Ahora los datos están en el formato que usa el clasificador y hay que convertir el texto sin procesar en funciones que sean útiles mediante la extracción de características.

### Extracción de características

Antes de poder entrenar el algoritmo se deben extraer las características de los textos es decir, reducir la cantidad de datos en un conjunto de atributos que el algoritmo pueda usar.

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer()
counts = count_vectorizer.fit_transform(data['text'].values)

In [6]:
counts

<55378x681407 sparse matrix of type '<class 'numpy.int64'>'
	with 9205824 stored elements in Compressed Sparse Row format>

El método fit_transform aprende el vocabulario del corpus y extrae características de conteo de palabras. Para obtener el texto del DataFrame, se accede a él como un diccionario y el método values para obtener la matriz Numpy necesaria

### Clasificación

Se va a usar un clasificador de Bayes Multinomial y lo entrenamos.

In [7]:
from sklearn.naive_bayes import MultinomialNB

classifier = MultinomialNB()
targets = data['class'].values
classifier.fit(counts, targets)

MultinomialNB()

Construimos dos ejemplos y predecimos. El de la Viagra debe ser Spam y el de Linux HAM.

In [8]:
examples = ['Free Viagra call today!', "I'm going to attend the Linux users group tomorrow."]
example_counts = count_vectorizer.transform(examples)
predictions = classifier.predict(example_counts)
predictions

array(['spam', 'ham'], dtype='<U4')

### Clasificación con Pipelines

In [9]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('vectorizer',  CountVectorizer()),
    ('classifier',  MultinomialNB()) ])

pipeline.fit(data['text'].values, data['class'].values)
pipeline.predict(examples)

array(['spam', 'ham'], dtype='<U4')

### Validación cruzada K-FOLD

In [10]:
from sklearn.model_selection import KFold, cross_validate, cross_val_score
from sklearn.metrics import confusion_matrix, f1_score
from scipy.stats import sem

# create a k-fold cross validation iterator of k=10 folds
cv = KFold(10, shuffle=True, random_state=33)

# by default the score used is the one returned by score method of the estimator (accuracy)
scores = cross_val_score(pipeline, data['text'].values, data['class'].values, cv=cv)
print(scores)

def mean_score(scores):
    return ("Mean score: {0:.3f} (+/- {1:.3f})").format(np.mean(scores), sem(scores))
print(mean_score(scores))

[0.93607801 0.93481401 0.93373059 0.93932828 0.93138317 0.93409173
 0.93661972 0.93373059 0.9328156  0.93317681]
Mean score: 0.935 (+/- 0.001)


Se puede observar que se obtiene una precisión de 0.935, que no está nada mal pero se puede mejorar.

## Exercise 3 - Automatic essay classification

As you have seen, we did not got great results in the previous notebook. You can try to improve them.

### Mejora de resultados

In [11]:
pipeline2 = Pipeline([
    ('count_vectorizer', CountVectorizer(ngram_range=(1, 2))),
    ('classifier',       MultinomialNB())
])

# create a k-fold cross validation iterator of k=10 folds
cv = KFold(10, shuffle=True, random_state=33)

# by default the score used is the one returned by score method of the estimator (accuracy)
scores2 = cross_val_score(pipeline2, data['text'].values, data['class'].values, cv=cv)
print(scores2)

def mean_score(scores2):
    return ("Mean score: {0:.3f} (+/- {1:.3f})").format(np.mean(scores2), sem(scores2))
print(mean_score(scores2))

[0.97670639 0.97526183 0.97345612 0.97833153 0.97291441 0.97381726
 0.97273384 0.97345612 0.97634098 0.97579917]
Mean score: 0.975 (+/- 0.001)


En la primera opción de mejora se ha utilizado un recuento de n-gramas en lugar de recuentos de palabras. Al Count Vector le hemos dado un rango de n-gramas que puede formar de 1 o 2 palabras es decir, unigramas y bigramas. 
Realizando la validación cruzada obtenemos una precisión de 0.975 por lo que observamos que funciona mejor que el recuento de palabras aunque aumenta el tiempo computacional.

In [12]:
from sklearn.feature_extraction.text import TfidfTransformer

pipeline3 = Pipeline([
    ('count_vectorizer',   CountVectorizer(ngram_range=(1,  2))),
    ('tfidf_transformer',  TfidfTransformer()),
    ('classifier',         MultinomialNB())
])

# create a k-fold cross validation iterator of k=10 folds
cv = KFold(10, shuffle=True, random_state=33)

# by default the score used is the one returned by score method of the estimator (accuracy)
scores3 = cross_val_score(pipeline3, data['text'].values, data['class'].values, cv=cv)
print(scores3)

def mean_score(scores3):
    return ("Mean score: {0:.3f} (+/- {1:.3f})").format(np.mean(scores3), sem(scores3))
print(mean_score(scores3))

[0.99097147 0.99133261 0.98952691 0.98988805 0.9907909  0.99133261
 0.99223546 0.99024919 0.99042803 0.99277587]
Mean score: 0.991 (+/- 0.000)


En este caso además de añadir el recuento por n-gramas de 1 o 2, se ha utilizado otro vectorizador, el TFIDF que convierte los recuentos en frecuencias y también se reduce el ruido de las características reduciendo el peso de las palabras que son muy comunes en todo el corpus usando la frecuencia inversa. 
Por lo tanto, usamos el recuento de n-gramas en el Count Vector y a continuación un TFIDF Vector. Así en la validación cruzada se obtiene una precisión de 0.991 que es mejor que las anteriores pruebas.

In [13]:
from sklearn.naive_bayes import BernoulliNB

pipeline4 = Pipeline([
    ('count_vectorizer',   CountVectorizer(ngram_range=(1, 2))),
    ('classifier',         BernoulliNB(binarize=0.0)) ])

# create a k-fold cross validation iterator of k=10 folds
cv = KFold(10, shuffle=True, random_state=33)

# by default the score used is the one returned by score method of the estimator (accuracy)
scores4 = cross_val_score(pipeline4, data['text'].values, data['class'].values, cv=cv)
print(scores4)

def mean_score(scores4):
    return ("Mean score: {0:.3f} (+/- {1:.3f})").format(np.mean(scores4), sem(scores4))
print(mean_score(scores4))


[0.95973276 0.95666306 0.9588299  0.96280246 0.9557602  0.9631636
 0.95846876 0.96099675 0.96387936 0.9604479 ]
Mean score: 0.960 (+/- 0.001)


En este caso, se ha utilizado el recuento por n-gramas de rango 1 y 2 pero además, se ha cambiado el clasificador a uno del tipo Bernoulli y se observa que la precisión obtenida tras la validación cruzada es peor que las otras mejoras pero mejor que la normal.

### Conclusiones

Nos quedariamos con la mejora que incluye n-gramas y TFIDF Vector ya que es la que mayor precisión obtiene tras realizar la validación cruzada con un valor de 0.991

## Licence

The notebook is freely licensed under under the [Creative Commons Attribution Share-Alike license](https://creativecommons.org/licenses/by/2.0/).  

© Carlos A. Iglesias, Universidad Politécnica de Madrid.